In [1]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import numpy as np
from math import ceil

from simulation import run_simulation
from graph_create import reduce_providers_simplest, powerlaw_dist_time, watts_strogatz, erdos_renyi, configuration_model
import util

In [2]:
init_notebook_mode(connected=True)

# Constants
num_vertices = 100
regularity = 4

In [3]:
def plot_utils(social_utils, plot_title):
    fig = go.Figure()

    #Add averages trace
    p_vals = [ p for (p, uts) in social_utils ]
    ut_vals = [ np.average(uts) for (p, uts) in social_utils ]
    fig.add_trace(go.Scatter(x=p_vals,
        y=ut_vals, mode='lines+markers', hoverinfo='skip',
        name='avg social welfare'))
    
    #Add individual points trace
    indiv_p_vals = []
    indiv_ut_vals = []
    for (p, uts) in social_utils:
        indiv_ut_vals.extend(uts)
        indiv_p_vals.extend([p] * len(uts))
    fig.add_trace(go.Scatter(x=indiv_p_vals,
        y=indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.7,
        marker=dict(color='LightSkyBlue'), name='social welfare per p'))
    
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title='Average social welfare (normalized)'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [4]:
def run_watts_strogatz(num_vertices, k, plaw_resources=False):
    for r in np.linspace(100, 0, 10, endpoint=False):
        for beta in np.linspace(1, 0, 10, endpoint=False):
            ws_social_utils = []
            ws_graph_diams = []
            
            cm_social_utils = []
            cm_graph_diams = []
            for p in np.linspace(1, 0, 10, endpoint=False):
                ws_utils = []
                cm_utils = []
                num_iter = 1
                for i in range(num_iter):
                    ring_lat = watts_strogatz(num_vertices, k, beta, p, p, r)
                    
                    degree_seq = [ vtx.degree for vtx in ring_lat.vertices ]

                    ws_graph_diams.append(util.calc_diameter(ring_lat))

                    reduce_providers_simplest(ring_lat)

                    if plaw_resources:
                        powerlaw_dist_time(ring_lat, 2)

                    sim_g, sim_utils = run_simulation(ring_lat)

                    #Get global social welfare at end of simulation normalized by size
                    ws_utils.append(sum(sim_utils[-1]) / len(ring_lat.vertices))
                    
                    #Run configuration model using WS degree seq
                    config_model = configuration_model(num_vertices, degree_seq, p, r)
                    cm_graph_diams.append(util.calc_diameter(config_model))
                    reduce_providers_simplest(config_model)
                    if plaw_resources:
                        powerlaw_dist_time(config_model, 2)
                        
                    csim_g, csim_utils = run_simulation(config_model)
                    cm_utils.append(sum(csim_utils[-1]) / len(config_model.vertices))
                
                ws_social_utils.append((p, ws_utils))
                cm_social_utils.append((p, cm_utils))

            plot_title_fmt = 'Average social welfare vs p (r={0}, beta={1}, avg diam={2})'
            ws_avg_diam = round(sum(ws_graph_diams) / len(ws_graph_diams), 2)
            ws_plot_title = plot_title_fmt.format(r, round(beta, 2), ws_avg_diam)
            plot_utils(ws_social_utils, ws_plot_title)
            
            cm_avg_diam = round(sum(cm_graph_diams) / len(cm_graph_diams), 2)
            cm_plot_title = plot_title_fmt.format(r, round(beta, 2), cm_avg_diam)
            plot_utils(cm_social_utils, cm_plot_title)

In [5]:
run_watts_strogatz(num_vertices, regularity)

KeyboardInterrupt: 